# Notebook for loading one ROSETTA/ICE lidar file
This uses the laspy package

See https://pythonhosted.org/laspy/tut_part_1.html for details 

Import packages. 
If you dont already have these, 
'pip install laspy' works well in Pangeo and on my local machine. (also 'conda laspy' will work)

In [82]:
import laspy.file as File
import xarray as xr

### Load the file

In [10]:
inFile = File("LASfiles/AN04_F1010_20171127_130056_LIDAR_VQ580_Record77.las", mode='r')


### the data come as sclaed and offset integers
correct for this to get true elevations and positions

In [87]:
lon = inFile.header.scale[0] * inFile.X + inFile.header.offset[0]
lat = inFile.header.scale[1] * inFile.Y + inFile.header.offset[1]
elevation = inFile.header.scale[2] * inFile.Z + inFile.header.offset[2]


### look at the names of the variables

In [93]:
for spec in inFile.point_format:
    print(spec.name)

X
Y
Z
intensity
flag_byte
classification_flags
classification_byte
user_data
scan_angle
pt_src_id
gps_time


### You can either write one variabale to an xarray.DataArray...
(http://xarray.pydata.org/en/stable/data-structures.html#dataarray)

In [94]:
# create the DataArray
da = xr.DataArray(elevation, coords={
    "lon": ("number", lon),
    "lat": ("number", lat)},
    dims=["number"])
# display the DataArray
da

<xarray.DataArray (number: 18872583)>
array([-52.72175, -52.72775, -52.72575, ..., -52.39625, -52.38575,
       -52.359  ])
Coordinates:
    lon      (number) float64 166.3 166.3 166.3 166.3 ... 166.8 166.8 166.8
    lat      (number) float64 -77.85 -77.85 -77.85 ... -77.85 -77.85 -77.85
Dimensions without coordinates: number

### ...or write multiple variables to an xarray.DataSet
(http://xarray.pydata.org/en/stable/data-structures.html#dataset)

In [95]:
ds = xr.Dataset(
    {
        "elevation": (["number"], elevation),
        "intensity": (["number"], inFile.intensity),
        "gps_time": (["number"], inFile.gps_time),
    },
    coords={
         "lon": ("number", lon),
        "lat": ("number", lat),
    },
)
# Display the dataset we just made
ds

<xarray.Dataset>
Dimensions:    (number: 18872583)
Coordinates:
    lon        (number) float64 166.3 166.3 166.3 166.3 ... 166.8 166.8 166.8
    lat        (number) float64 -77.85 -77.85 -77.85 ... -77.85 -77.85 -77.85
Dimensions without coordinates: number
Data variables:
    elevation  (number) float64 -52.72 -52.73 -52.73 ... -52.4 -52.39 -52.36
    intensity  (number) uint16 36306 31358 32603 34242 ... 24870 23297 17268
    gps_time   (number) float64 4.684e+04 4.684e+04 ... 4.697e+04 4.697e+04